In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
x_train = pd.read_csv('./data/train.csv', header=None)
y_train = pd.read_csv('./data/trainLabels.csv', header=None)
test = pd.read_csv('./data/test.csv', header=None)
x_train.shape[0], y_train.shape[0], test.shape[0]

(1000, 1000, 9000)

In [2]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [3]:
param_grid = {'n_estimators': [150], 'min_samples_split': [.01], 'min_samples_leaf': [1], 'max_depth': [3]}
clf = GradientBoostingClassifier(random_state=7, max_depth=1)

grid_search = GridSearchCV(clf, param_grid, scoring="f1", n_jobs=-1, verbose=1)
grid_result = grid_search.fit(x_train, y_train)

print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\christine\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:   11.1s finished
C:\Users\christine\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Accuracy: 0.884650 using {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 0.01, 'n_estimators': 150}


In [4]:
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'], 
                                          min_samples_leaf=grid_result.best_params_['min_samples_leaf'], 
                                          min_samples_split=grid_result.best_params_['min_samples_split'], 
                                          n_estimators=grid_result.best_params_['n_estimators'])
clf_bestparam.fit(x_train, y_train)
print(metrics.f1_score(y_val, clf_bestparam.predict(x_val)))

C:\Users\christine\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8704663212435233


In [5]:
ans = pd.DataFrame({'Id': list(range(1, 9001)), 'Solution': clf_bestparam.predict(test)}).set_index('Id')
ans.to_csv('./data/ans.csv')